Very simple file that takes the expansion.bin by Shawn and splits it into train.bin and val.bin for Delphi training!

In [1]:
import torch
import numpy
from pathlib import Path
import numpy as np

data_path = "./expansion.bin"
data_path = Path(data_path)
assert data_path.exists(), "Data file does not exist."

np_data = np.memmap(str(data_path), mode="r", dtype=np.uint32).reshape(-1, 3)


batch_ids = np_data[:, 0].astype(int)
time_points = np_data[:, 1].astype(int)
event_types = np_data[:, 2].astype(int)

In [3]:
train_perc = 0.7
val_perc = 0.2
test_perc = 0.1


sequences = []

batch_ids_np = np.asarray(batch_ids)
change_idcs = np.flatnonzero(batch_ids_np[1:] != batch_ids_np[:-1]) + 1
change_idcs = np.concatenate([np.array([0]), change_idcs, np.array([len(batch_ids_np)])])

num_batches = len(change_idcs) - 1


In [ ]:
# %%
# Sequences always start with male/female. Filter illegal sequences.

male_token = 2
female_token = 3

legal_start_stop = []
for s, e in zip(change_idcs[:-1], change_idcs[1:]):
    rel_seq = event_types[s:e]

    if rel_seq[0] == male_token or rel_seq[0] == female_token:
        legal_start_stop.append((s, e))


array([   3,  603,  661,  664,  266,  485,  292,  757,  428,  834,  794,
        574,  890,   22,  518,  602,  792,  534,  587,  531,  629,  514,
        499,  505,  509,  869,  532,  533,  622,  173,  273,  497, 1269])

In [ ]:
num_train_batches = int(x=train_perc * num_batches)
num_val_batches = int(val_perc * num_batches)
num_test_batches = num_batches - (num_train_batches + num_val_batches)


In [3]:
train_batchess = {
    "batch_ids": batch_ids[change_idcs[0] : change_idcs[num_train_batches]],
    "time_points": time_points[change_idcs[0] : change_idcs[num_train_batches]],
    "event_types": event_types[change_idcs[0] : change_idcs[num_train_batches]],
}

val_batchess = {
    "batch_ids": batch_ids[change_idcs[num_train_batches] : change_idcs[num_train_batches + num_val_batches]],
    "time_points": time_points[change_idcs[num_train_batches] : change_idcs[num_train_batches + num_val_batches]],
    "event_types": event_types[change_idcs[num_train_batches] : change_idcs[num_train_batches + num_val_batches]],
}

test_batchess = {
    "batch_ids": batch_ids[change_idcs[num_train_batches + num_val_batches] :],
    "time_points": time_points[change_idcs[num_train_batches + num_val_batches] :],
    "event_types": event_types[change_idcs[num_train_batches + num_val_batches] :],
}

In [5]:
for n, b in zip(["train", "val", "test"], [train_batchess, val_batchess, test_batchess]):
    out_path = f"{n}.bin"
    bin_content = np.stack(
        [b["batch_ids"].astype(np.uint32), b["time_points"].astype(np.uint32), b["event_types"].astype(np.uint32)],
        axis=1,
    ).reshape(-1)

    mm = np.memmap(out_path, mode="w+", dtype=np.uint32, shape=bin_content.shape)
    mm[:] = bin_content[:]
    del mm
